In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

# Rainfall Distribution Partitioned by Day

# VERY HEAVY RAINFALL

In [6]:
def post(post_name, low, high, sheet, row, col, dist = None, type_ = None, aligned = None):
    post_df = pd.read_csv('rainfall_df_' + post_name + '.csv')
    post_df.sort_values(by = ['Date', 'Time'], ascending = [1, 1], inplace = True)
    post_df['Rainfall (mm)'] = post_df['Rainfall (mm)'].astype('float')
    post_df['Date'] = pd.to_datetime(post_df['Date']).dt.date
    daily = post_df.groupby('Date')['Rainfall (mm)'].sum()
    if low >= 1: daily = daily[daily >= low]
    else: daily = daily[(daily >= low) & (daily < high)]
    post_df = post_df[post_df['Date'].isin(daily.index)]
    if len(post_df) == 0: return
    pivot = post_df.pivot(index='Date', columns='Time', values='Rainfall (mm)')
    pivot.columns = [int(str(c).split(':')[0]) for c in pivot.columns]
    
    if aligned:
        post_df = post_df[post_df['Rainfall (mm)'] > 0]
        post_df["hour_in_event"] = post_df.groupby("Date").cumcount() + 1
        pivot = post_df.pivot(index = "Date", columns = "hour_in_event", values = "Rainfall (mm)")
    
    if post_name == 'pasirjaya': pivot.drop(pivot.loc[datetime.date(2022, 3, 2):datetime.date(2022, 4, 9)].index, inplace = True)
    duration, rainfall, intensity = pivot.count(axis = 1), pivot.sum(axis = 1), pivot.mean(axis = 1)
    pivot['duration'], pivot['rainfall'], pivot['intensity'] = duration, rainfall, intensity
    
    if dist:
        col = 0
        for duration in range(1, pivot['duration'].max() + 1):
            percent = pivot[pivot['duration'] == duration]
            if percent.empty: continue
            total = percent.iloc[:, 0:duration].sum(axis = 1)
            percent = percent.iloc[:, 0:duration].div(total, axis = 0) * 100
            mean, count = percent.mean(), percent.count()
            percent.loc['count'], percent.loc[(str(duration) + ' Jam')] = count, mean
            #percent.to_excel(writer, sheet_name = '%' + sheet, index = True, startrow = row, startcol = col)
            #col += percent.shape[1] + 3
            
            #display(percent.loc[[str(duration) + " Jam"]])
            percent.loc[[(str(duration) + ' Jam')]].to_excel(writer, sheet_name = 'summary', index = True, header = False, startrow = row, startcol = 3)
            #percent.to_excel(writer, sheet_name = '%' + sheet, index = True, startrow = row, startcol = col)
            #col += percent.shape[1] + 3
            row += 1
        row += 4
        return percent.loc[[str(duration) + " Jam"]]
    
    mean_all = pivot.mean()
    mean_rain = pivot[pivot > 0].mean()
    #if not mean_all.empty:
    #    pivot.loc['true_mean'] = mean_all
    #    pivot.loc['rainy_only'] = mean_rain
    
    
    def get_span_duration(row):
        rain_hours = row[row > 0]
        if rain_hours.empty:
            return np.nan
        return rain_hours.index[-1] - rain_hours.index[0] + 1
    
    def get_first_rain_hour(row):
        for hour in range(24):
            if row[hour] > 0: return hour
        return None  # If no rain that day
    
    #pivot['duration_cont'] = pivot[list(range(24))].apply(get_span_duration, axis=1)
    #pivot['intensity_cont'] = pivot['total_rain']/pivot['duration_cont']
    pivot['first_rain_hour'] = pivot.apply(get_first_rain_hour, axis=1)
    
    if type_ == 'duration_freq':
        if 'true_mean' in pivot.index: pivot.drop('true_mean', inplace = True)
        if 'rainy_only' in pivot.index: pivot.drop('rainy_only', inplace = True)
        if 'true_normalized' in pivot.index: pivot.drop('true_normalized', inplace = True)
        if 'rainy_normalized' in pivot.index: pivot.drop('rainy_normalized', inplace = True)
        display(pivot)
        duration_freq = pivot.groupby('duration')['duration'].count().to_frame()
        duration_freq.sort_index(inplace = True)
        duration_freq.rename(columns = {'duration': 'frequency'}, inplace = True)
        duration_freq['%'] = duration_freq['frequency']/duration_freq['frequency'].sum()
        display(duration_freq)
        #duration_freq.to_excel(writer, sheet_name = sheet, index = True, startrow = row, startcol = col)
        return
    
    elif type_ == 'start_hour':
        if 'true_mean' in pivot.index: pivot.drop('true_mean', inplace = True)
        if 'rainy_only' in pivot.index: pivot.drop('rainy_only', inplace = True)
        if 'true_normalized' in pivot.index: pivot.drop('true_normalized', inplace = True)
        if 'rainy_normalized' in pivot.index: pivot.drop('rainy_normalized', inplace = True)

        start_of_rain = pivot.groupby('first_rain_hour')['first_rain_hour'].count().to_frame()
        start_of_rain = start_of_rain.reindex(range(24), fill_value = 0)
        start_of_rain['%'] = start_of_rain['first_rain_hour']/start_of_rain['first_rain_hour'].sum()
        start_of_rain.to_excel(writer, sheet_name = sheet, index = True, startrow = row, startcol = col)
        return

    elif type_ == 'non_continuous':
        def realign_rainfall_span(row):
            rain = row[list(range(24))]
            rain_hours = rain[rain > 0]
            start = rain_hours.index[0]
            end = rain_hours.index[-1]
            span = rain.loc[start:end].values
            shifted = list(span) + [np.nan] * (24 - len(span))
            return pd.Series(shifted)
    
        aligned = pivot.apply(realign_rainfall_span, axis=1)
        if dist: aligned = aligned.iloc[:-4]
        else: aligned = aligned.iloc[:-2]
        aligned.columns = range(1,25)
        aligned['duration'] = aligned.count(axis = 1)
        
        return aligned#.iloc[:-10]
    
    elif type_ == 'intensity':
        post_df['datetime'] = pd.to_datetime(post_df['Date'].astype(str) + ' ' + post_df['Time'])
        post_df['is_raining'] = post_df['Rainfall (mm)'] > 0
        post_df['time_diff'] = post_df['datetime'].diff()
        post_df['new_event'] = (post_df['is_raining']) & (
            (post_df['time_diff'] > pd.Timedelta(hours=1)) |
            (~post_df['is_raining'].shift(1, fill_value=False))
        )
        post_df['event_id'] = post_df['new_event'].cumsum()
        post_df.loc[~post_df['is_raining'], 'event_id'] = 0
        post_df = post_df[post_df['event_id'] > 0]
        post_df['hour_in_event'] = post_df.groupby(['Date', 'event_id']).cumcount() + 1
        if aligned: pivot = post_df.pivot(index = ['Date', 'event_id'], columns = 'hour_in_event', values = 'Rainfall (mm)')
        else: pivot = post_df.pivot(index = ['Date', 'event_id'], columns = 'Time', values = 'Rainfall (mm)')
        if post_name == 'pasirjaya': pivot.drop(pivot.loc[datetime.date(2022, 3, 2):datetime.date(2022, 4, 9)].index, inplace = True)
        pivot.columns = [int(str(c).split(':')[0]) for c in pivot.columns]
        duration, rainfall, intensity = pivot.count(axis = 1), pivot.sum(axis = 1), pivot.mean(axis = 1)
        pivot['duration'], pivot['rainfall'], pivot['intensity'] = duration, rainfall, intensity
        
        intensity = pivot[['duration', 'rainfall', 'intensity']]
        bins = [0, 10, 20, 30, 40, 50, 100, float('inf')]
        labels = ['0 - 10', '10 - 20', '20 - 30', '30 - 40', '40 - 50', '50 - 100', '> 100']
        intensity['category'] = pd.cut(intensity['intensity'], bins = bins, labels = labels, right = False)
        count = intensity.groupby('category')['category'].count().to_frame()
        count['%'] = count['category']/count['category'].sum()
        
        intensity.to_excel(writer, sheet_name = 'int_count', index = True, startrow = row, startcol = col)
        count.to_excel(writer, sheet_name = 'int_count', index = True, startrow = len(intensity) + 5, startcol = col)
    
    # Rename columns to Hour_1 .. Hour_24
    #aligned_rain.columns = [f'Hour_{i+1}' for i in range(24)]

    
    if dist: pivot.to_excel(writer, sheet_name = '%' + sheet , index = True, startrow = row, startcol = col)
    else: pivot.to_excel(writer, sheet_name = sheet, index = True, startrow = row, startcol = col)
    return pivot

def filter_(df, mode):
    columns = list(range(1, mode + 1)) + ['duration']
    df = df[df['duration'] == mode].loc[:, columns]
    count, total, mean = df.count(), df.sum(), df.mean()
    stats = pd.DataFrame([count, total, mean], index=['count', 'total', 'mean'])
    df = pd.concat([df, stats])
    return df

## General

In [11]:
with pd.ExcelWriter(
    'dist_vheavy.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 0
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 100, None, i, 3, col)

## Aligned

In [12]:
with pd.ExcelWriter(
    'dist_vheavy.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 31
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 100, None, i, 3, col, aligned = True)

## Distribution

In [15]:
with pd.ExcelWriter(
    'dist_vheavy.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    row = 3
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        df = post(i, 100, None, i, row, 3, dist = True, aligned = True)
        if df is None: continue
        row += df.shape[1] + 4

## Duration Frequency

In [72]:
with pd.ExcelWriter(
    'dist_vheavy.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 0
    for i in ['cawang']:#, 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 100, None, 'duration_freq2', 3, col, type_ = 'duration_freq', aligned = True)
        col += 4

hour_in_event,1,2,3,4,5,6,7,8,9,10,11,12,13,14,duration,rainfall,intensity
Date,,,,,,,,,,,,,,,,,
2018-10-23,28.0,46.5,7.0,20.5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,105.0,21.000000
2018-10-27,64.5,1.0,157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,222.5,74.166667
2018-11-03,152.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,152.0,152.000000
2018-11-06,27.0,132.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,159.0,79.500000
2018-11-09,159.0,8.0,75.5,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,334.5,83.625000
2018-12-09,94.0,11.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,105.5,52.750000
2019-01-25,2.0,15.0,2.0,4.0,6.5,38.5,60.0,213.5,NaN,NaN,NaN,NaN,NaN,NaN,8,341.5,42.687500
2019-01-30,47.5,2.0,47.5,36.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,133.5,33.375000
2019-02-11,4.0,2.0,2.0,275.5,263.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,547.0,109.400000


,frequency,%
duration,,
1,1,0.027778
2,3,0.083333
3,5,0.138889
4,6,0.166667
5,6,0.166667
6,4,0.111111
7,1,0.027778
8,2,0.055556
9,2,0.055556


## Start of Hour Frequency

In [7]:
with pd.ExcelWriter(
    'dist_vheavy.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 30
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 100, None, 'duration_freq2', 3, col, type_ = 'start_hour')
        if i in ['cawang', 'pamulang']: col += 5
        else: col += 4

## Intensity Frequency

In [13]:
with pd.ExcelWriter(
    'dist_vheavy.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 0
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 100, None, i, 3, col, type_ = 'intensity')
        col += 7

# INTENSE RAINFALL

In [5]:
def post(post_name, low, high, sheet, row, col, dist = None, type_ = None, aligned = None, intense = None):
    post_df = pd.read_csv('rainfall_df_' + post_name + '.csv')
    post_df.sort_values(by = ['Date', 'Time'], ascending = [1, 1], inplace = True)
    post_df['Rainfall (mm)'] = post_df['Rainfall (mm)'].astype('float')
    post_df['Date'] = pd.to_datetime(post_df['Date']).dt.date
    daily = post_df.groupby('Date')['Rainfall (mm)'].sum()
    daily = daily[daily >= low]
    post_df = post_df[post_df['Date'].isin(daily.index)]
    if len(post_df) == 0: return
    post_df['datetime'] = pd.to_datetime(post_df['Date'].astype(str) + ' ' + post_df['Time'])
    post_df['is_raining'] = post_df['Rainfall (mm)'] > 0
    post_df['time_diff'] = post_df['datetime'].diff()
    post_df['new_event'] = (post_df['is_raining']) & (
        (post_df['time_diff'] > pd.Timedelta(hours=1)) |
        (~post_df['is_raining'].shift(1, fill_value=False))
    )
    post_df['event_id'] = post_df['new_event'].cumsum()
    post_df.loc[~post_df['is_raining'], 'event_id'] = 0
    post_df = post_df[post_df['event_id'] > 0]
    post_df['hour_in_event'] = post_df.groupby(['Date', 'event_id']).cumcount() + 1
    if aligned: pivot = post_df.pivot(index = ['Date', 'event_id'], columns = 'hour_in_event', values = 'Rainfall (mm)')
    else: pivot = post_df.pivot(index = ['Date', 'event_id'], columns = 'Time', values = 'Rainfall (mm)')
    if post_name == 'pasirjaya': pivot.drop(pivot.loc[datetime.date(2022, 3, 2):datetime.date(2022, 4, 9)].index, inplace = True)
    pivot.columns = [int(str(c).split(':')[0]) for c in pivot.columns]
    duration, rainfall, intensity = pivot.count(axis = 1), pivot.sum(axis = 1), pivot.mean(axis = 1)
    pivot['duration'], pivot['rainfall'], pivot['intensity'] = duration, rainfall, intensity
    if intense: pivot = pivot[pivot['intensity'] >= intense]
    if i in ['cawang', 'pamulang', 'pasarbaru', 'pasirjaya']: pivot = pivot[pivot['intensity'] >= 10]
    else: pivot = pivot[pivot['intensity'] >= 10]
    if dist:
        col = 0
        for duration in range(1, pivot['duration'].max() + 1):
            percent = pivot[pivot['duration'] == duration]
            if percent.empty: continue
            total = percent.iloc[:, 0:duration].sum(axis = 1)
            percent = percent.iloc[:, 0:duration].div(total, axis = 0) * 100
            mean, count = percent.mean(), percent.count()
            percent.loc[('count', 0), :], percent.loc[(str(duration) + ' Jam', 0), :] = count, mean
            percent.to_excel(writer, sheet_name = '%' + sheet, index = True, startrow = row, startcol = col)
            col += percent.shape[1] + 3
            
            #display(percent.loc[[str(duration) + " Jam"]])
            #percent.loc[[(str(duration) + ' Jam', 0)], :].to_excel(writer, sheet_name = 'summary', index = True, header = False, startrow = row, startcol = 3)
            #percent.to_excel(writer, sheet_name = '%' + sheet, index = True, startrow = row, startcol = col)
            #col += percent.shape[1] + 3
            #row += 1
        #row += 4
        return percent.loc[[str(duration) + " Jam"]]
        
    def get_first_rain_hour(row):
        for hour in range(24):
            if row[hour] > 0: return hour
        return None  # If no rain that day
    
    pivot['first_rain_hour'] = pivot.apply(get_first_rain_hour, axis=1)
    
    #mean_all = pivot.mean(axis = 0)
    #mean_rain = pivot[pivot > 0].mean()
    #pivot.loc[('mean', 0),:] = mean_all
    #pivot.loc[('rainy_only', '')] = mean_rain
    
    #if dist:
    #    pivot.loc['true_normalized'] = mean_all/mean_all.sum() * 100
    #    pivot.loc['rainy_normalized'] = mean_rain/mean_rain.sum() * 100
    
    if type_ == 'duration_freq':
        duration_freq = pivot.groupby('duration')['duration'].count().to_frame()
        duration_freq.sort_index(inplace = True)
        duration_freq.rename(columns = {'duration': 'frequency'}, inplace = True)
        duration_freq['%'] = duration_freq['frequency']/duration_freq['frequency'].sum()
        duration_freq.to_excel(writer, sheet_name = sheet, index = True, startrow = row, startcol = col)
        return
    
    elif type_ == 'start_hour':
        start_of_rain = pivot.groupby('first_rain_hour')['first_rain_hour'].count().to_frame()
        start_of_rain = start_of_rain.reindex(range(24), fill_value = 0)
        start_of_rain['%'] = start_of_rain['first_rain_hour']/start_of_rain['first_rain_hour'].sum()
        start_of_rain.to_excel(writer, sheet_name = sheet, index = True, startrow = row, startcol = col)
        return

    elif type_ == 'non_continuous':
        def realign_rainfall_span(row):
            rain = row[list(range(24))]
            rain_hours = rain[rain > 0]
            start = rain_hours.index[0]
            end = rain_hours.index[-1]
            span = rain.loc[start:end].values
            shifted = list(span) + [np.nan] * (24 - len(span))
            return pd.Series(shifted)
    
        aligned = pivot.apply(realign_rainfall_span, axis=1)
        if dist: aligned = aligned.iloc[:-4]
        else: aligned = aligned.iloc[:-2]
        aligned.columns = range(1,25)
        aligned['duration'] = aligned.count(axis = 1)
        
        return aligned#.iloc[:-10]
    
    elif type_ == 'intensity_freq':
        intensity = pivot[['duration', 'rainfall', 'intensity']]
        bins = [0, 10, 20, 30, 40, 50, 100, float('inf')]
        labels = ['0 - 10', '10 - 20', '20 - 30', '30 - 40', '40 - 50', '50 - 100', '> 100']
        intensity['category'] = pd.cut(intensity['intensity'], bins = bins, labels = labels, right = False)
        count = intensity.groupby('category')['category'].count().to_frame()
        count['%'] = count['category']/count['category'].sum()
        
        intensity.to_excel(writer, sheet_name = 'int_count', index = True, startrow = row, startcol = col)
        count.to_excel(writer, sheet_name = 'int_count', index = True, startrow = len(intensity) + 5, startcol = col)
        return
    
    if dist: pivot.to_excel(writer, sheet_name = '%' + sheet , index = True, startrow = row, startcol = col)
    else: pivot.to_excel(writer, sheet_name = sheet, index = True, startrow = row, startcol = col)
    return pivot

def filter_(df, mode):
    columns = list(range(1, mode + 1)) + ['duration']
    df = df[df['duration'] == mode].loc[:, columns]
    count, total, mean = df.count(), df.sum(), df.mean()
    stats = pd.DataFrame([count, total, mean], index=['count', 'total', 'mean'])
    df = pd.concat([df, stats])
    
    return df

In [136]:
post('cawang', 1, None, 'cawang', 3, 0, dist = True, aligned = True)

1
Date       event_id       
2018-10-27 6         100.0
           8         100.0
2018-11-01 14        100.0
2018-11-03 17        100.0
2018-12-04 35        100.0
2019-02-06 103       100.0
2019-03-18 130       100.0
2019-05-05 154       100.0
2019-05-11 160       100.0
2020-03-15 284       100.0
2021-01-29 479       100.0
2021-03-20 546       100.0
2021-06-18 601       100.0
2021-09-30 631       100.0
2021-10-30 648       100.0
2021-10-31 651       100.0
2021-11-01 654       100.0
2021-12-14 690       100.0
2022-02-05 749       100.0
2022-02-16 760       100.0
2022-04-11 787       100.0
2022-05-20 817       100.0
2022-09-07 864       100.0
2022-10-04 879       100.0
2023-02-19 996       100.0
2023-03-04 1023      100.0
2023-04-08 1048      100.0
2023-04-19 1057      100.0
2023-04-24 1061      100.0
2023-12-28 1134      100.0
2024-03-04 1189      100.0
2024-10-16 1198      100.0
2024-11-16 1211      100.0
2024-11-26 1217      100.0
2024-12-05 1225      100.0
2025-01-17 1256      100.0

AttributeError: 'MergedCell' object attribute 'value' is read-only

## Distribution

In [8]:
with pd.ExcelWriter(
    'dist_intense.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 0
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        df = post(i, 1, None, i, 3, col, dist = True, aligned = True)
        #row += df.shape[1] + 4

## General

In [58]:
with pd.ExcelWriter(
    'dist_intense.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 30
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 1, None, i, 3, col)

## Aligned

In [56]:
with pd.ExcelWriter(
    'dist_intense.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 0
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 1, None, i, 3, col, aligned = True)

## Intensity Frequency

In [32]:
with pd.ExcelWriter(
    'dist_intense.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 0
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 1, None, i, 3, col, type_ = 'intensity_freq')
        col += 7

## Duration Frequency

In [6]:
with pd.ExcelWriter(
    'dist_intense.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 104
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 1, None, 'int_count', 3, col, type_ = 'duration_freq')
        col += 4

   ## Start of the Hour Frequency

In [10]:
with pd.ExcelWriter(
    'dist_intense.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 0
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 1, None, 'start_hour', 3, col, type_ = 'start_hour')
        if i in ['cawang', 'pamulang']: col += 5
        else: col += 4

# GLOBAL

In [13]:
def post(post_name, low, high, sheet, row, col, dist = None, type_ = None, aligned = None):
    post_df = pd.read_csv('rainfall_df_' + post_name + '.csv')
    post_df.sort_values(by = ['Date', 'Time'], ascending = [1, 1], inplace = True)
    post_df['Rainfall (mm)'] = post_df['Rainfall (mm)'].astype('float')
    post_df['Date'] = pd.to_datetime(post_df['Date']).dt.date
    daily = post_df.groupby('Date')['Rainfall (mm)'].sum()
    daily = daily[daily >= low]
    post_df = post_df[post_df['Date'].isin(daily.index)]
    if len(post_df) == 0: return
    pivot = post_df.pivot(index='Date', columns='Time', values='Rainfall (mm)')
    pivot.columns = [int(str(c).split(':')[0]) for c in pivot.columns]
    
    if aligned:
        post_df = post_df[post_df['Rainfall (mm)'] > 0]
        post_df["hour_in_event"] = post_df.groupby("Date").cumcount() + 1
        pivot = post_df.pivot(index = "Date", columns = "hour_in_event", values = "Rainfall (mm)")
    
    if post_name == 'pasirjaya': pivot.drop(pivot.loc[datetime.date(2022, 3, 2):datetime.date(2022, 4, 9)].index, inplace = True)
    duration, rainfall, intensity = pivot.count(axis = 1), pivot.sum(axis = 1), pivot.mean(axis = 1)
    pivot['duration'], pivot['rainfall'], pivot['intensity'] = duration, rainfall, intensity
    
    if dist:
        col = 0
        for duration in range(1, pivot['duration'].max() + 1):
            percent = pivot[pivot['duration'] == duration]
            if percent.empty: continue
            total = percent.iloc[:, 0:duration].sum(axis = 1)
            percent = percent.iloc[:, 0:duration].div(total, axis = 0) * 100
            mean, count = percent.mean(), percent.count()
            percent.loc['count'], percent.loc['mean'] = count, mean
            percent.loc[str(duration) + " Jam"] = mean
            display(percent.loc[[str(duration) + " Jam"]])
            percent.loc[[str(duration) + " Jam"]].to_excel(writer, sheet_name = 'summary', index = True, header = False, startrow = row, startcol = 3)
            #percent.to_excel(writer, sheet_name = '%' + sheet, index = True, startrow = row, startcol = col)
            #col += percent.shape[1] + 3
            row += 1
        row += 4
        return percent.loc[[str(duration) + " Jam"]]
    
    mean_all = pivot.mean()
    mean_rain = pivot[pivot > 0].mean()
    #if not mean_all.empty:
    #    pivot.loc['true_mean'] = mean_all
    #    pivot.loc['rainy_only'] = mean_rain
    
    
    def get_span_duration(row):
        rain_hours = row[row > 0]
        if rain_hours.empty:
            return np.nan
        return rain_hours.index[-1] - rain_hours.index[0] + 1
    
    def get_first_rain_hour(row):
        for hour in range(24):
            if row[hour] > 0: return hour
        return None  # If no rain that day
    
    #pivot['duration_cont'] = pivot[list(range(24))].apply(get_span_duration, axis=1)
    #pivot['intensity_cont'] = pivot['total_rain']/pivot['duration_cont']
    pivot['first_rain_hour'] = pivot.apply(get_first_rain_hour, axis=1)
    
    if type_ == 'duration_freq':
        if 'true_mean' in pivot.index: pivot.drop('true_mean', inplace = True)
        if 'rainy_only' in pivot.index: pivot.drop('rainy_only', inplace = True)
        if 'true_normalized' in pivot.index: pivot.drop('true_normalized', inplace = True)
        if 'rainy_normalized' in pivot.index: pivot.drop('rainy_normalized', inplace = True)
        duration_freq = pivot.groupby('duration')['duration'].count().to_frame()
        duration_freq.sort_index(inplace = True)
        duration_freq.rename(columns = {'duration': 'frequency'}, inplace = True)
        duration_freq['%'] = duration_freq['frequency']/duration_freq['frequency'].sum()
        duration_freq.to_excel(writer, sheet_name = sheet, index = True, startrow = row, startcol = col)
        return
    
    elif type_ == 'start_hour':
        if 'true_mean' in pivot.index: pivot.drop('true_mean', inplace = True)
        if 'rainy_only' in pivot.index: pivot.drop('rainy_only', inplace = True)
        if 'true_normalized' in pivot.index: pivot.drop('true_normalized', inplace = True)
        if 'rainy_normalized' in pivot.index: pivot.drop('rainy_normalized', inplace = True)

        start_of_rain = pivot.groupby('first_rain_hour')['first_rain_hour'].count().to_frame()
        start_of_rain = start_of_rain.reindex(range(24), fill_value = 0)
        start_of_rain['%'] = start_of_rain['first_rain_hour']/start_of_rain['first_rain_hour'].sum()
        start_of_rain.to_excel(writer, sheet_name = sheet, index = True, startrow = row, startcol = 40 + col)
        return

    elif type_ == 'non_continuous':
        def realign_rainfall_span(row):
            rain = row[list(range(24))]
            rain_hours = rain[rain > 0]
            start = rain_hours.index[0]
            end = rain_hours.index[-1]
            span = rain.loc[start:end].values
            shifted = list(span) + [np.nan] * (24 - len(span))
            return pd.Series(shifted)
    
        aligned = pivot.apply(realign_rainfall_span, axis=1)
        if dist: aligned = aligned.iloc[:-4]
        else: aligned = aligned.iloc[:-2]
        aligned.columns = range(1,25)
        aligned['duration'] = aligned.count(axis = 1)
        
        return aligned#.iloc[:-10]
    
    elif type_ == 'intensity':
        post_df['datetime'] = pd.to_datetime(post_df['Date'].astype(str) + ' ' + post_df['Time'])
        post_df['is_raining'] = post_df['Rainfall (mm)'] > 0
        post_df['time_diff'] = post_df['datetime'].diff()
        post_df['new_event'] = (post_df['is_raining']) & (
            (post_df['time_diff'] > pd.Timedelta(hours=1)) |
            (~post_df['is_raining'].shift(1, fill_value=False))
        )
        post_df['event_id'] = post_df['new_event'].cumsum()
        post_df.loc[~post_df['is_raining'], 'event_id'] = 0
        post_df = post_df[post_df['event_id'] > 0]
        post_df['hour_in_event'] = post_df.groupby(['Date', 'event_id']).cumcount() + 1
        if aligned: pivot = post_df.pivot(index = ['Date', 'event_id'], columns = 'hour_in_event', values = 'Rainfall (mm)')
        else: pivot = post_df.pivot(index = ['Date', 'event_id'], columns = 'Time', values = 'Rainfall (mm)')
        if post_name == 'pasirjaya': pivot.drop(pivot.loc[datetime.date(2022, 3, 2):datetime.date(2022, 4, 9)].index, inplace = True)
        pivot.columns = [int(str(c).split(':')[0]) for c in pivot.columns]
        duration, rainfall, intensity = pivot.count(axis = 1), pivot.sum(axis = 1), pivot.mean(axis = 1)
        pivot['duration'], pivot['rainfall'], pivot['intensity'] = duration, rainfall, intensity
        
        intensity = pivot[['duration', 'rainfall', 'intensity']]
        bins = [0, 10, 20, 30, 40, 50, 100, float('inf')]
        labels = ['0 - 10', '10 - 20', '20 - 30', '30 - 40', '40 - 50', '50 - 100', '> 100']
        bins2 = [0, 1, 5, 10, 20, 50, 100, float('inf')]
        labels2 = ['0 - 1', '1 - 5', '5 - 10', '10 - 20', '20 - 50', '50 - 100', '> 100']
        intensity['category'] = pd.cut(intensity['intensity'], bins = bins, labels = labels, right = False)
        intensity['category2'] = pd.cut(intensity['intensity'], bins = bins2, labels = labels2, right = False)
        count = intensity.groupby('category')['category'].count().to_frame()
        count['%'] = count['category']/count['category'].sum()
        count2 = intensity.groupby('category2')['category2'].count().to_frame()
        count2['%'] = count2['category2']/count2['category2'].sum()
        
        intensity.to_excel(writer, sheet_name = 'int_count2', index = True, startrow = row, startcol = col)
        count.to_excel(writer, sheet_name = 'int_count2', index = True, startrow = row, startcol = 65 + col/8*4)
        count2.to_excel(writer, sheet_name = 'int_count2', index = True, startrow = 14, startcol = 65 + col/8*4)
        return
    
    # Rename columns to Hour_1 .. Hour_24
    #aligned_rain.columns = [f'Hour_{i+1}' for i in range(24)]
    
    if dist: pivot.to_excel(writer, sheet_name = '%' + sheet , index = True, startrow = row, startcol = col)
    else: pivot.to_excel(writer, sheet_name = sheet, index = True, startrow = row, startcol = col)
    return pivot

def filter_(df, mode):
    columns = list(range(1, mode + 1)) + ['duration']
    df = df[df['duration'] == mode].loc[:, columns]
    count, total, mean = df.count(), df.sum(), df.mean()
    stats = pd.DataFrame([count, total, mean], index=['count', 'total', 'mean'])
    df = pd.concat([df, stats])
    return df

## General

In [60]:
with pd.ExcelWriter(
    'dist_global.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 0
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 1, None, i, 3, col)

## Aligned

In [62]:
with pd.ExcelWriter(
    'dist_global.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 31
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 1, None, i, 3, col, aligned = True)

## Distribution

In [18]:
with pd.ExcelWriter(
    'dist_global.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    row = 2
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        df = post(i, 1, None, i, row, 0, dist = True, aligned = True)
        row += df.shape[1] + 4

hour_in_event,1
Date,
1 Jam,100.0


hour_in_event,1,2
Date,,
2 Jam,51.609207,48.390793


hour_in_event,1,2,3
Date,,,
3 Jam,36.156862,41.521827,22.321311


hour_in_event,1,2,3,4
Date,,,,
4 Jam,31.353662,30.123737,23.09788,15.424721


hour_in_event,1,2,3,4,5
Date,,,,,
5 Jam,20.746543,34.512138,19.742789,16.661004,8.337526


...

hour_in_event,1,2,3,4,5,6,7,8,9,10
Date,,,,,,,,,,
10 Jam,3.861068,3.42952,9.209153,11.620945,3.912612,25.665545,3.42952,23.221187,9.753925,5.896523


hour_in_event,1,2,3,4,5,6,7,8,9,10,11
Date,,,,,,,,,,,
11 Jam,12.631045,8.525755,8.02109,14.66038,6.501039,17.711929,7.081214,5.897184,11.808516,4.591695,2.570154


hour_in_event,1,2,3,4,5,6,7,8,9,10,11,12
Date,,,,,,,,,,,,
12 Jam,3.21884,5.5487,8.113945,26.381462,8.208785,8.869235,12.061794,6.274281,4.663147,3.777595,9.895333,2.986882


hour_in_event,1,2,3,4,5,6,7,8,9,10,11,12,13
Date,,,,,,,,,,,,,
13 Jam,2.5,2.5,6.666667,6.666667,11.666667,2.5,23.333333,15.0,1.666667,3.333333,0.833333,8.333333,15.0


hour_in_event,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Date,,,,,,,,,,,,,,,,,,
18 Jam,8.333333,5.555556,1.851852,7.407407,6.481481,23.148148,5.555556,3.703704,0.925926,4.62963,3.703704,0.925926,0.925926,3.703704,2.777778,17.592593,1.851852,0.925926


## Duration Frequency

In [74]:
with pd.ExcelWriter(
    'dist_global.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 0
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 1, None, 'duration_freq2', 3, col, type_ = 'duration_freq', aligned = True)
        col += 4

## Start of Hour Frequency

In [5]:
with pd.ExcelWriter(
    'dist_global.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 0
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 1, None, 'start_hour', 3, col, type_ = 'start_hour')
        col += 4

## Intensity Frequency

In [91]:
with pd.ExcelWriter(
    'dist_global.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 0
    for i in ['cawang', 'awsciawi', 'situparigi', 'pamulang', 'pasarbaru', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 1, None, 'int_count', 3, col, type_ = 'intensity')
        col += 8

<br><br><br><br><br><br><br><br><br><br>
# MISC

In [37]:
with pd.ExcelWriter(
    'distribution4.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 30
    for i in ['cawang', 'pamulang', 'pasirjaya']:
        post(i, 200, None, 'duration_freq', 3, col, type_ = 'start_hour')
        col += 3

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,rain_hour,total_rain,intensity,duration_cont,intensity_cont,first_rain_hour
Date,,,,,,,,,,,,,,,,,,,,,
2018-10-27,0.0,0.0,64.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,157.0,0.0,3.0,222.5,74.166667,21,10.595238,2
2018-11-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,334.5,83.625000,6,55.750000,12
2019-01-25,0.0,2.0,15.0,2.0,0.0,0.0,4.0,6.5,38.5,60.0,...,0.0,0.0,0.0,0.0,8.0,341.5,42.687500,10,34.150000,1
2019-02-11,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,5.0,547.0,109.400000,18,30.388889,0
2021-10-28,14.0,36.0,33.0,52.0,17.5,57.5,17.0,10.0,30.5,2.0,...,0.0,0.0,0.0,0.0,14.0,271.5,19.392857,20,13.575000,0
2021-10-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,257.0,51.400000,10,25.700000,10
2025-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,218.5,72.833333,3,72.833333,13


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,rain_hour,total_rain,intensity,duration_cont,intensity_cont,first_rain_hour
Date,,,,,,,,,,,,,,,,,,,,,
2021-10-27,NaN,NaN,NaN,NaN,NaN,16.0,0.5,0.0,9.5,0.0,...,74.0,51.0,71.5,29.0,17.0,370.5,21.794118,19,19.500000,5
2021-10-28,18.5,11.0,64.5,59.0,43.5,60.5,10.5,0.5,0.0,0.5,...,0.0,0.0,0.0,0.0,16.0,342.5,21.406250,19,18.026316,0
2024-01-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,355.5,71.100000,7,50.785714,11


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,rain_hour,total_rain,intensity,duration_cont,intensity_cont,first_rain_hour
Date,,,,,,,,,,,,,,,,,,,,,
2022-03-12,6.0,6.0,6.0,6.0,6.0,6.0,5.5,6.0,6.0,6.0,...,6.0,6.0,6.0,6.0,24.0,214.0,8.916667,24,8.916667,0
2023-11-07,NaN,NaN,NaN,NaN,NaN,NaN,365.0,46.5,0.0,0.0,...,NaN,NaN,NaN,NaN,2.0,411.5,205.750000,2,205.750000,6


In [15]:
with pd.ExcelWriter(
    'distribution4.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    dist = True
    if dist == True: col = 29
    else: col = 0
    for i in ['cawang', 'pasarbaru', 'awsciawi', 'situparigi', 'pamulang', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        row = 1
        aligned = post(i, 200, None, i, 1, col, dist, type_ = 'non_continuous')
        if aligned is None: continue
        duration_count = aligned.groupby('duration')['duration'].count().sort_values(ascending = False).to_frame()
        for mode in aligned['duration'].mode().to_list():
            filtered = filter_(aligned, mode)
            filtered.to_excel(writer, sheet_name = '%' + i, index = True, startrow = row, startcol = 5)
            row += len(filtered) + 4
            
        aligned.to_excel(writer, sheet_name = i, index = True, startrow = 1, startcol = col)
        duration_count.to_excel(writer, sheet_name = '%' + i, index = True, startrow = 1, startcol = 0)
        #col += 4
    
    #mode = df.iloc[:-3]['duration'].mode().iloc[0]

In [39]:
with pd.ExcelWriter(
    'distribution4.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 0
    for i in ['cawang', 'pamulang', 'pasirjaya']:
        post(i, 200, None, 'duration_freq', 2, col, type_ = 'duration_freq')
        col += 4

In [44]:
with pd.ExcelWriter(
    'distribution1.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 30
    for i in ['cawang', 'pasarbaru', 'awsciawi', 'situparigi', 'pamulang', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 100, None, 'duration_freq2', 3, col, type_ = 'start_hour')
        col += 3

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,rain_hour,total_rain,intensity,duration_cont,intensity_cont,first_rain_hour
Date,,,,,,,,,,,,,,,,,,,,,
2018-10-23,28.0,46.5,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,105.0,21.000000,18,5.833333,0
2018-10-27,0.0,0.0,64.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,157.0,0.0,3.0,222.5,74.166667,21,10.595238,2
2018-11-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,152.0,152.000000,1,152.000000,17
2018-11-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,159.0,79.500000,2,79.500000,13
2018-11-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,334.5,83.625000,6,55.750000,12
2018-12-09,0.0,0.0,0.0,94.0,11.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,105.5,52.750000,2,52.750000,3
2019-01-25,0.0,2.0,15.0,2.0,0.0,0.0,4.0,6.5,38.5,60.0,...,0.0,0.0,0.0,0.0,8.0,341.5,42.687500,10,34.150000,1
2019-01-30,47.5,2.0,0.0,47.5,36.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,133.5,33.375000,5,26.700000,0
2019-02-11,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,5.0,547.0,109.400000,18,30.388889,0


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,rain_hour,total_rain,intensity,duration_cont,intensity_cont,first_rain_hour
Date,,,,,,,,,,,,,,,,,,,,,
2022-11-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,121.0,17.285714,7,17.285714,11
2022-11-17,7.0,32.0,24.0,8.0,4.0,5.0,4.0,3.0,3.0,4.0,...,0.0,0.0,0.0,0.0,12.0,101.0,8.416667,13,7.769231,0
2023-11-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,9.0,112.0,12.444444,11,10.181818,9
2023-12-25,0.0,0.0,0.0,4.0,10.0,0.0,1.0,0.0,0.0,0.0,...,100.0,16.0,0.0,0.0,7.0,135.0,19.285714,19,7.105263,3
2024-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13.0,5.0,4.0,2.0,11.0,174.0,15.818182,11,15.818182,13
2024-01-19,4.0,7.0,7.0,29.0,34.0,8.0,8.0,23.0,8.0,10.0,...,0.0,0.0,0.0,0.0,16.0,196.0,12.250000,17,11.529412,0
2024-01-27,0.0,0.0,0.0,0.0,0.0,0.0,10.0,6.0,4.0,18.0,...,0.0,4.0,0.0,0.0,10.0,164.0,16.400000,16,10.250000,6
2024-03-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,19.0,9.0,...,1.0,0.0,1.0,0.0,14.0,108.0,7.714286,16,6.750000,7
2024-04-12,0.0,97.0,31.0,4.0,1.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9.0,183.0,20.333333,17,10.764706,1


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,rain_hour,total_rain,intensity,duration_cont,intensity_cont,first_rain_hour
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,...,0.0,0.0,0.0,0.0,5.0,194.0,38.8,11,17.636364,9


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,rain_hour,total_rain,intensity,duration_cont,intensity_cont,first_rain_hour
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,146.5,48.833333,6,24.416667,13


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,rain_hour,total_rain,intensity,duration_cont,intensity_cont,first_rain_hour
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,36.5,10.0,5.5,2.0,17.5,30.5,20.0,48.0,16.0,4.0,...,0.0,0.0,0.0,0.5,13.0,192.0,14.769231,24,8.000000,0
2021-10-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.5,10.5,29.0,46.0,8.0,170.5,21.312500,15,11.366667,9
2021-10-27,NaN,NaN,NaN,NaN,NaN,16.0,0.5,0.0,9.5,0.0,...,74.0,51.0,71.5,29.0,17.0,370.5,21.794118,19,19.500000,5
2021-10-28,18.5,11.0,64.5,59.0,43.5,60.5,10.5,0.5,0.0,0.5,...,0.0,0.0,0.0,0.0,16.0,342.5,21.406250,19,18.026316,0
2021-10-31,3.0,1.5,2.0,5.0,3.5,1.0,4.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,12.0,101.0,8.416667,16,6.312500,0
2022-04-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,69.5,2.0,1.0,2.0,7.0,107.5,15.357143,12,8.958333,12
2022-11-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,105.5,21.100000,5,21.100000,15
2023-11-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,27.5,1.5,1.5,2.5,8.0,104.0,13.000000,10,10.400000,14
2023-12-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,150.5,50.166667,3,50.166667,15


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,rain_hour,total_rain,intensity,duration_cont,intensity_cont,first_rain_hour
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,122.0,24.40,5,24.400000,15
2024-11-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,129.0,43.00,4,32.250000,15
2024-11-27,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,122.0,15.25,17,7.176471,3


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,rain_hour,total_rain,intensity,duration_cont,intensity_cont,first_rain_hour
Date,,,,,,,,,,,,,,,,,,,,,
2020-03-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,105.5,52.750000,2,52.750000,18
2020-05-10,4.5,2.0,4.5,39.0,8.0,25.0,23.0,8.5,2.5,0.0,...,0.0,0.0,0.0,0.0,12.0,153.0,12.750000,16,9.562500,0
2020-09-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,156.5,39.125000,4,39.125000,15
2022-03-02,6.0,6.0,5.5,6.0,5.0,5.5,6.0,6.0,6.0,6.0,...,6.0,6.0,6.0,5.0,24.0,139.5,5.812500,24,5.812500,0
2022-03-03,4.5,5.5,4.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,6.0,6.0,6.0,6.0,24.0,137.5,5.729167,24,5.729167,0
2022-03-04,6.0,6.0,6.0,6.0,6.0,5.5,4.5,5.0,6.0,6.0,...,6.0,6.0,6.0,6.0,24.0,140.5,5.854167,24,5.854167,0
2022-03-05,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,5.5,5.0,5.5,6.0,24.0,143.5,5.979167,24,5.979167,0
2022-03-06,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,6.0,6.0,5.5,6.5,24.0,143.0,5.958333,24,5.958333,0
2022-03-07,6.0,6.0,6.0,6.0,6.0,6.0,6.0,5.5,6.0,5.5,...,6.0,5.5,6.0,6.0,24.0,141.0,5.875000,24,5.875000,0


In [40]:
post('pasirjaya', 100, None, 'first_hour', 2, col)

,first_rain_hour,%
first_rain_hour,,
0,43,0.728814
1,0,0.000000
2,0,0.000000
3,1,0.016949
4,0,0.000000
5,0,0.000000
6,2,0.033898
7,0,0.000000
8,2,0.033898


In [43]:
with pd.ExcelWriter(
    'distribution1.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    col = 0
    for i in ['cawang', 'pasarbaru', 'awsciawi', 'situparigi', 'pamulang', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        post(i, 100, None, 'duration_freq2', 3, col, type_ = 'duration_freq')
        col += 4

In [9]:
with pd.ExcelWriter(
    'distribution.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    for i in ['cawang', 'pasarbaru', 'awsciawi', 'situparigi', 'pamulang', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        if i == 'lengkongbarang': continue # Very heavy rainfall in Lengkong Barang doesnt exist
        post(i, 100, None, i, 1, 3*32)

In [38]:
with pd.ExcelWriter(
    'distribution.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    for i in ['cawang', 'pasarbaru', 'awsciawi', 'situparigi', 'pamulang', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        for idx, (low, high) in enumerate([(5, 20), (20, 50), (50, 100), (100, None)]):
            post(i, low, high, i, 1, idx*32)

In [5]:
for i in ['cawang']:#, 'pasarbaru', 'awsciawi', 'situparigi', 'pamulang', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
    for idx, (low, high) in enumerate([(5, 20), (20, 50), (50, 100), (100, None)]):
        post(i, low, high, i, 1, idx*31)

In [5]:
def post2(post_name, sheet, type_ = None, mode = None):
    post_df = pd.read_csv('rainfall_df_' + post_name + '.csv')
    post_df.sort_values(by = ['Date', 'Time'], ascending = [1, 1], inplace = True)
    post_df['Rainfall (mm)'] = post_df['Rainfall (mm)'].astype('float')
    post_df['Date'] = pd.to_datetime(post_df['Date']).dt.date
    post_df['datetime'] = pd.to_datetime(post_df['Date'].astype(str) + ' ' + post_df['Time'])
    post_df['is_raining'] = post_df['Rainfall (mm)'] > 0
    post_df['time_diff'] = post_df['datetime'].diff()

    post_df['new_event'] = (post_df['is_raining']) & (
        (post_df['time_diff'] > pd.Timedelta(hours=1)) |
        (~post_df['is_raining'].shift(1, fill_value=False))
    )
    post_df['event_id'] = post_df['new_event'].cumsum()
    post_df.loc[~post_df['is_raining'], 'event_id'] = 0
    event_total = post_df.groupby('event_id')['Rainfall (mm)'].sum()
    valid = event_total[event_total >= 100].index
    post_df = post_df[post_df['event_id'].isin(valid)]
    post_df['hour_in_event'] = post_df.groupby('event_id').cumcount()
    
    event_dates = post_df.groupby('event_id')['datetime'].agg(['min', 'max']).reset_index()
    event_dates['date_start'] = event_dates['min'].dt.date
    event_dates['date_end'] = event_dates['max'].dt.date
    event_dates = event_dates[['event_id', 'date_start', 'date_end']]

    post_df['day_offset'] = post_df.groupby('event_id')['datetime'].transform(
        lambda x: (x.dt.floor('D') - x.dt.floor('D').min()).dt.days + 1
    )
    post_df['hour_label'] = post_df['datetime'].dt.hour.astype(str) + '_' + post_df['day_offset'].astype(str)
    if post_df.empty: return

    if type_ == 'daily':
        final_df = post_df.pivot(index='Date', columns='Time', values='Rainfall (mm)')
        final_df.columns = [int(str(c).split(':')[0]) for c in final_df.columns]
        count, total, mean = final_df.count(), final_df.sum(), final_df.mean()
        final_df.loc['count'] = count
        final_df.loc['total'] = total
        final_df.loc['mean'] = mean
    elif type_ == 'global': 
        pivot = post_df.pivot(index='event_id', columns='hour_label', values='Rainfall (mm)')
        final_df = event_dates.merge(pivot, on='event_id')
        meta_cols = ['event_id', 'date_start', 'date_end']
        data_cols = sorted([col for col in final_df.columns if col not in meta_cols], key=lambda x: (int(x.split('_')[1]), int(x.split('_')[0])))
    
        final_df = final_df[meta_cols + data_cols]
        duration = (final_df.drop(columns = meta_cols).notna()).sum(axis=1)
        total = final_df.drop(columns = meta_cols).sum(axis=1)
        mean = final_df.drop(columns = meta_cols).mean(axis = 1)
        final_df['duration'] = duration
        final_df['total_rain'] = total
        final_df['mean'] = mean
        #final_df.loc['count'] = (final_df.notna()).sum(axis = 1)
    else:
        post_df['relative'] = post_df.groupby('event_id')['datetime'].transform(
            lambda x: (x - x.min()).dt.total_seconds() // 3600 + 1
        )
        post_df['relative'] = post_df['relative'].astype(int)
        final_df = post_df.pivot(index='event_id', columns = 'relative', values = 'Rainfall (mm)')
        if type_ == 'filter' or type_ == '% filter':
            final_df['duration'] = (final_df.notna()).sum(axis=1)
            final_df = final_df[final_df['duration'] == mode].drop(columns = 'duration')
        
        if type_ == '% dist' or type_ == '% filter':
            final_df = final_df.div(final_df.sum(axis=1), axis = 0) * 100
            mean = final_df.mean()
            count = final_df.count()
            final_df.loc['count'] = count
            final_df.loc['mean'] = mean
            final_df.loc['mean_norm'] = mean/mean.sum() * 100
            total = final_df.sum(axis = 1)
            final_df['duration'] = (final_df.notna()).sum(axis=1)
            final_df['total'] = total
        elif type_ == 'filter' or type_ == 'dist':
            count, total, mean = final_df.count(), final_df.sum(), final_df.mean()
            final_df.loc['count'] = count
            final_df.loc['mean'] = mean
            final_df.loc['total'] = total
            duration, total, mean = (final_df.notna()).sum(axis=1), final_df.sum(axis=1), final_df.mean(axis = 1)
            final_df['duration'] = duration
            final_df['total_rain'] = total
            final_df['mean'] = mean
            
        elif type_ == 'start_hour':
            start_of_rain = post_df[post_df['relative'] == 1]
            return start_of_rain.groupby('Time')['Time'].count()

    #if dist: pivot.to_excel(writer, sheet_name = '%' + sheet , index = True, startrow = row, startcol = col)
    #else: pivot.to_excel(writer, sheet_name = sheet, index = True, startrow = row, startcol = col)
    
    return start_of_rain

Time
01:00:00    1
02:00:00    2
03:00:00    1
06:00:00    1
10:00:00    3
11:00:00    2
12:00:00    1
13:00:00    5
14:00:00    2
15:00:00    1
16:00:00    4
17:00:00    1
20:00:00    1
21:00:00    2
22:00:00    1
23:00:00    2
Name: Time, dtype: int64

In [20]:
with pd.ExcelWriter(
    'distribution2 - Copy.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    startcol = 0
    for i in ['cawang', 'pasarbaru', 'awsciawi', 'situparigi', 'pamulang', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        df = post2(i, 'start', 'start_hour')
        if df is None: continue
        df = df.to_frame()
        df.loc['name'] = i
        display(df)
        df.to_excel(writer, sheet_name = 'start', startrow = 1, startcol = startcol, index = True)
        startcol += 3
        print(startcol)

,Time
Time,
01:00:00,1
02:00:00,2
03:00:00,1
06:00:00,1
10:00:00,3
11:00:00,2
12:00:00,1
13:00:00,5
14:00:00,2


3


,Time
Time,
00:00:00,1
01:00:00,1
06:00:00,1
07:00:00,1
11:00:00,1
13:00:00,1
20:00:00,1
22:00:00,3
23:00:00,1


6


,Time
Time,
09:00:00,1
name,awsciawi


...

15


,Time
Time,
13:00:00,1
15:00:00,2
name,rancabungur


18


,Time
Time,
00:00:00,1
03:00:00,1
06:00:00,2
08:00:00,1
12:00:00,3
13:00:00,3
14:00:00,1
15:00:00,3
16:00:00,1


21


In [106]:
with pd.ExcelWriter(
    'distribution2.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    for i in ['cawang', 'pasarbaru', 'awsciawi', 'situparigi', 'pamulang', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        startcol = 0
        for j in ['global', 'daily']:
            df = post2(i, i, j)
            if df is None: continue
            df.to_excel(writer, sheet_name = i, startrow = 1, startcol = startcol)
            startcol += len(df.columns) + 4
            print(startcol)

45
73
49
77
12
18
12
18
45
73
17
28
958
986


In [109]:
with pd.ExcelWriter(
    'distribution2.xlsx',
    engine = 'openpyxl',
    mode = 'a',
    if_sheet_exists = 'overlay'
) as writer:
    for i in ['cawang', 'pasarbaru', 'awsciawi', 'situparigi', 'pamulang', 'lengkongbarang', 'rancabungur', 'pasirjaya']:
        startcol = 0
        for j in ['dist', '% dist']:
            df = post2(i, i, j)
            if df is None: continue
            df.to_excel(writer, sheet_name = '%' + i, startrow = 1, startcol = startcol)
            startcol += len(df.columns) + 4
            
        if df is not None: mode = df.iloc[:-3]['duration'].mode().iloc[0]
        print(mode)
        for j in ['filter', '% filter']:
            filter_df = post2(i, i, j, mode)
            if filter_df is None: continue
            filter_df.to_excel(writer, sheet_name = '%' + i, startrow = 1, startcol = startcol)
            startcol += len(filter_df.columns) + 4

2
3
2
2
3
3
2
4
